In [1]:
import sys
sys.path.append('..')
import os
from os.path import expanduser

import lcatools

In [2]:
from lcatools.providers.ecoinvent_spreadsheet import EcoinventSpreadsheet

In [3]:
ECOINVENT_VERSION = '3.2'

SYSTEM_MODELS = ['undefined', 'apos', 'consequential', 'cut-off']

datapath = os.path.join(expanduser('~'), 'Dropbox', 'data', 'Ecoinvent', ECOINVENT_VERSION)


In [4]:
ei = dict()
for model in SYSTEM_MODELS:
    pub_file = os.path.join(datapath, 'activity_overview_for_users_' + ECOINVENT_VERSION + '_' + model + '.xlsx')
    ei[model] = EcoinventSpreadsheet(pub_file, internal=False, version=ECOINVENT_VERSION)
    

In [6]:
for model in SYSTEM_MODELS:
    ei[model].load_all()

Reading elementary exchanges ...
Reading intermediate exchanges ...
0 new quantity entities added (20 total)
Handling elementary exchanges [public spreadsheet]
0 new flow entities added (6767 total)
Handling intermediate exchanges [public spreadsheet]
0 new flow entities added (6767 total)
Handling activities...
Reading activity overview ...
0 new process entities added (13307 total)
Reading elementary exchanges ...
Reading intermediate exchanges ...
0 new quantity entities added (20 total)
Handling elementary exchanges [public spreadsheet]
0 new flow entities added (6767 total)
Handling intermediate exchanges [public spreadsheet]
0 new flow entities added (6767 total)
Handling activities...
Reading activity overview ...
0 new process entities added (11420 total)
Reading elementary exchanges ...
Reading intermediate exchanges ...
0 new quantity entities added (20 total)
Handling elementary exchanges [public spreadsheet]
0 new flow entities added (6767 total)
Handling intermediate excha

In [18]:
for k, v in ei.items():
    fname = 'ecoinvent_%s_%s_xlsx.json' % (ECOINVENT_VERSION, k)
    v.write_to_file(fname, exchanges=True)

In [7]:
ei

{'apos': <lcatools.providers.ecoinvent_spreadsheet.EcoinventSpreadsheet at 0x7f420d073a20>,
 'consequential': <lcatools.providers.ecoinvent_spreadsheet.EcoinventSpreadsheet at 0x7f420d073a90>,
 'cut-off': <lcatools.providers.ecoinvent_spreadsheet.EcoinventSpreadsheet at 0x7f420d073b00>,
 'undefined': <lcatools.providers.ecoinvent_spreadsheet.EcoinventSpreadsheet at 0x7f420d0739e8>}